In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler as SDS
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
pwd = os.getcwd()
filepath_1 = os.path.join(pwd, "Datasets\\cosmicclassifierTraining.csv")
filepath_2 = os.path.join(pwd, "Datasets\\cosmicclassifierTest.csv")
filepath_3 = os.path.join(pwd, "Datasets\\submission.csv")


filepath_1

'D:\\ml_projects\\Cosmic_Classsifier\\Datasets\\cosmicclassifierTraining.csv'

In [3]:
cosmic_train_df = pd.read_csv(filepath_1)
X_test = pd.read_csv(filepath_2)
Y_test = pd.read_csv(filepath_3)

In [4]:
cosmic_train_df.head()

,Atmospheric Density,Surface Temperature,Gravity,Water Content,Mineral Abundance,Orbital Period,Proximity to Star,Magnetic Field Strength,Radiation Levels,Atmospheric Composition Index,Prediction
0,0.472806,NaN,-0.313872,-2.089299,-0.152201,-0.885649,0.900105,NaN,Category_6,0.692907,5.0
1,4.180154,-1.157515,2.430956,-1.595850,-3.188678,-0.609434,-0.199828,Category_9,Category_9,NaN,0.0
2,-0.129008,1.621592,-0.785741,2.081196,-1.413796,-0.095152,-3.502577,NaN,Category_8,-0.677182,4.0
3,-3.122000,-2.299818,1.072092,0.353524,-0.192529,2.917067,-1.972329,NaN,Category_11,0.109429,1.0
4,-1.459426,2.890268,0.148757,-0.804439,0.494875,0.044910,-0.438796,Category_6,Category_10,0.407941,9.0


In [5]:
cosmic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Atmospheric Density            57016 non-null  float64
 1   Surface Temperature            56968 non-null  float64
 2   Gravity                        57016 non-null  float64
 3   Water Content                  56923 non-null  float64
 4   Mineral Abundance              57079 non-null  float64
 5   Orbital Period                 57003 non-null  float64
 6   Proximity to Star              57055 non-null  float64
 7   Magnetic Field Strength        56942 non-null  object 
 8   Radiation Levels               56979 non-null  object 
 9   Atmospheric Composition Index  57058 non-null  float64
 10  Prediction                     56961 non-null  float64
dtypes: float64(9), object(2)
memory usage: 5.0+ MB


In [6]:
print(cosmic_train_df["Prediction"].unique())

[ 5.  0.  4.  1.  9.  2. nan  3.  6.  7.  8.]


In [7]:
simple_imputer = SimpleImputer(strategy="most_frequent")
cosmic_train_df[["Prediction", "Magnetic Field Strength", "Radiation Levels"]] = simple_imputer.fit_transform(cosmic_train_df[["Prediction", "Magnetic Field Strength", "Radiation Levels"]])

In [8]:
numeric_cols = cosmic_train_df.select_dtypes(include=['number']).columns
iter_imputer = IterativeImputer(random_state=42)
cosmic_train_df[numeric_cols] = iter_imputer.fit_transform(cosmic_train_df[numeric_cols])

In [9]:
cosmic_train_df.head(10)

,Atmospheric Density,Surface Temperature,Gravity,Water Content,Mineral Abundance,Orbital Period,Proximity to Star,Magnetic Field Strength,Radiation Levels,Atmospheric Composition Index,Prediction
0,0.472806,-0.000758,-0.313872,-2.089299,-0.152201,-0.885649,0.900105,Category_9,Category_6,0.692907,5.0
1,4.180154,-1.157515,2.430956,-1.595850,-3.188678,-0.609434,-0.199828,Category_9,Category_9,-0.000395,0.0
2,-0.129008,1.621592,-0.785741,2.081196,-1.413796,-0.095152,-3.502577,Category_9,Category_8,-0.677182,4.0
3,-3.122000,-2.299818,1.072092,0.353524,-0.192529,2.917067,-1.972329,Category_9,Category_11,0.109429,1.0
4,-1.459426,2.890268,0.148757,-0.804439,0.494875,0.044910,-0.438796,Category_6,Category_10,0.407941,9.0
5,0.749979,1.064353,2.510823,0.105772,1.203825,1.355501,0.000339,Category_10,Category_9,0.551453,2.0
6,-2.971646,-0.648251,-0.915859,0.255504,-0.537165,-2.072251,1.355523,Category_8,Category_7,1.876232,1.0
7,1.981474,-1.347445,-0.483752,-3.365811,2.690204,1.199946,-1.907013,Category_12,Category_5,0.443053,1.0
8,-3.306354,-0.316716,-0.431264,0.389815,-1.961216,-1.510182,0.538593,Category_8,Category_7,0.934055,1.0
9,-1.188687,1.775882,2.666702,0.434578,0.205058,-0.000999,-1.545261,Category_10,Category_9,-1.973059,4.0


In [10]:
cat_encoder = OHE(handle_unknown='ignore', sparse_output=False)
cosmic_cat = cosmic_train_df[["Magnetic Field Strength", "Radiation Levels"]]
encoded_array = cat_encoder.fit_transform(cosmic_cat)
encoded_df = pd.DataFrame(
    encoded_array, 
    columns=cat_encoder.get_feature_names_out(["Magnetic Field Strength", "Radiation Levels"]), 
    index=cosmic_train_df.index
)
cosmic_train_df.drop(columns=["Magnetic Field Strength", "Radiation Levels"], inplace=True)
cosmic_train_df = pd.concat([cosmic_train_df, encoded_df], axis=1)
cosmic_cat_test = X_test[["Magnetic Field Strength", "Radiation Levels"]]
encoded_array_2 = cat_encoder.transform(cosmic_cat_test)
encoded_df = pd.DataFrame(
    encoded_array_2, 
    columns=cat_encoder.get_feature_names_out(["Magnetic Field Strength", "Radiation Levels"]), 
    index=X_test.index
)
X_test.drop(columns=["Magnetic Field Strength", "Radiation Levels"], inplace=True)
X_test = pd.concat([X_test, encoded_df], axis=1)

In [11]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
X = cosmic_train_df.drop(columns=["Prediction"])
y = cosmic_train_df["Prediction"]
for train_idx, test_idx in split.split(X, y):
    X_train = X.iloc[train_idx]
    X_valid = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[test_idx]

In [13]:
scaler = SDS()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_valid.columns, index=X_valid.index)
X_test_scaled  = pd.DataFrame(X_test_scaled,  columns=X_test.columns,  index=X_test.index)

In [15]:
X_train_scaled.to_csv("X_train.csv", index=False)
X_valid_scaled.to_csv("X_valid.csv", index=False)
X_test_scaled.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_valid.to_csv("y_valid.csv", index=False)